In [3]:
Sarcasm_Path = '/sarcasm_headlines.csv'

In [4]:
import pandas as pd

def read_and_view_csv(Sarcasm_Path):
    sarcasm_headlines = pd.read_csv(Sarcasm_Path)
    sarcasm_headlines.dropna(inplace=True)
    print("Shape= ", sarcasm_headlines.shape)

    print("Columns names:", sarcasm_headlines.columns.tolist())
    return sarcasm_headlines
sarcasm_headlines= read_and_view_csv(Sarcasm_Path)

Shape=  (26600, 3)
Columns names: ['headline', 'is_sarcastic', 'Cleaned_Headlines']


In [5]:
sarcasm_headlines = sarcasm_headlines[['is_sarcastic','Cleaned_Headlines']] #take cleaned headlines content and the output variables.

In [6]:
sarcasm_headlines

,is_sarcastic,Cleaned_Headlines
0,0,former versace store clerk sues secret black c...
1,0,roseanne revival catches thorny political mood...
2,1,mom starting fear son web series closest thing...
3,1,boehner wants wife listen come alternative deb...
4,0,j k rowling wishes snape happy birthday magica...
...,...,...
26597,0,american politics moral free fall
26598,0,america best hikes
26599,0,reparations obama
26600,0,israeli ban targeting boycott supporters raise...


In [7]:
import warnings as WARN
WARN.filterwarnings('ignore')

In [8]:
sarcasm_headlines['is_sarcastic'].value_counts()

,count
is_sarcastic,
0,14949
1,11651


In [9]:
X = sarcasm_headlines['Cleaned_Headlines']
y = sarcasm_headlines['is_sarcastic']

In [10]:
!pip install fasttext

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

Tokenizer_Var = Tokenizer()
Tokenizer_Var.fit_on_texts(sarcasm_headlines['Cleaned_Headlines'])


In [12]:
#embedding Matrix
import fasttext

def fasttext_Embedder(fasttext_model_path):
    model = fasttext.load_model(fasttext_model_path)
    print(f"Loaded FastText model.")
    return model

fasttext_File = '/cc.en.300.bin'
fasttext_mod = fasttext_Embedder(fasttext_File)


import numpy as np
embed_dim = 300

embedding_matrix = np.zeros((len(Tokenizer_Var.word_index) + 1, embed_dim))

for word, input in Tokenizer_Var.word_index.items():
    embedding_matrix[input] = fasttext_mod.get_word_vector(word)


Loaded FastText model.


In [13]:
##  padding sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences as pad_seq

max_seq_len = 300
X_tok = Tokenizer_Var.texts_to_sequences(sarcasm_headlines['Cleaned_Headlines'])

X = pad_seq(X_tok, maxlen=max_seq_len)
X.shape


(26600, 300)

In [14]:
from sklearn.decomposition import PCA

decomposer_PCA = PCA(n_components = 10)
X = decomposer_PCA.fit_transform(X)
X.shape

(26600, 10)

In [15]:
X

array([[ 1.11435591e+03, -3.87261951e+03, -3.11722431e+03, ...,
         3.11096126e+03,  1.16216608e+04, -4.21923619e+03],
       [-3.37201073e+03,  3.99565700e+03, -3.14506315e+03, ...,
         4.03939249e+03, -3.46417772e+03, -3.09713541e+02],
       [ 2.55357526e+03, -4.99774613e+03, -1.12807252e+03, ...,
        -8.10251444e+01, -4.96616419e+02, -1.02907735e+02],
       ...,
       [-2.57106467e+03, -4.86021408e+03,  2.55470022e+03, ...,
        -2.21849146e+02, -1.41435682e+02, -8.00611338e+01],
       [-7.68421663e+02,  2.62269970e-01, -2.22774783e+03, ...,
         6.94159288e+02, -8.13725743e+02, -1.28748902e+02],
       [ 2.10777268e+04, -7.33497302e+03,  8.49217379e+03, ...,
        -2.36216175e+02, -1.71031564e+02, -7.15543426e+01]])

In [16]:
# scaling the input
from sklearn.preprocessing import MinMaxScaler as scaler
scaler_var = scaler()

X = scaler_var.fit_transform(X)
X

array([[0.20112599, 0.25955035, 0.50018365, ..., 0.40288449, 0.58972895,
        0.11071495],
       [0.09154073, 0.41438105, 0.49964715, ..., 0.42550762, 0.18778248,
        0.21076661],
       [0.23628077, 0.23741029, 0.53851777, ..., 0.32510522, 0.26685006,
        0.21605914],
       ...,
       [0.11110486, 0.24011663, 0.60949067, ..., 0.32167376, 0.27631348,
        0.21664383],
       [0.15513673, 0.33576031, 0.51732528, ..., 0.34399418, 0.25840101,
        0.21539782],
       [0.68875696, 0.19141871, 0.72391525, ..., 0.32132368, 0.27552493,
        0.21686153]])

In [17]:
from sklearn.model_selection import train_test_split as split

def train_test(X, y):

    X_trn, X_tst, y_trn, y_tst = split(X, y, test_size=0.4, random_state=1)
    X_val, X_tst, y_val, y_tst = split(X_tst, y_tst, test_size=0.5, random_state=1)

    return X_trn, X_tst, X_val, y_trn, y_tst, y_val

X_trn, X_tst, X_val, y_trn, y_tst, y_val = train_test(X,y)

print("Training data size:", len(X_trn))
print("Testing data size:", len(X_tst))
print("validation data size:", len(X_val))


Training data size: 15960
Testing data size: 5320
validation data size: 5320


In [18]:
X_trn

array([[0.1375967 , 0.34640311, 0.54207342, ..., 0.30090099, 0.27693797,
        0.21524588],
       [0.14658484, 0.41286101, 0.43114844, ..., 0.32033398, 0.2932936 ,
        0.27060872],
       [0.10115068, 0.29880858, 0.64125208, ..., 0.33336666, 0.26230842,
        0.21518001],
       ...,
       [0.15795056, 0.35369085, 0.48814575, ..., 0.32928771, 0.26927565,
        0.21597781],
       [0.03412547, 0.2963973 , 0.56927554, ..., 0.45518988, 0.53308476,
        0.11893647],
       [0.12531387, 0.30611136, 0.62085432, ..., 0.31867818, 0.27420893,
        0.21593591]])

In [19]:
X_tst

array([[0.10720872, 0.41076628, 0.51779145, ..., 0.31900554, 0.27096724,
        0.21525721],
       [0.07965383, 0.27254606, 0.57887075, ..., 0.33447667, 0.27067729,
        0.21372791],
       [0.13482698, 0.3936866 , 0.60901847, ..., 0.31398548, 0.27353833,
        0.21507917],
       ...,
       [0.19138362, 0.62649817, 0.42980136, ..., 0.30304664, 0.27492489,
        0.2115836 ],
       [0.19419701, 0.56768553, 0.43074625, ..., 0.2467004 , 0.27709749,
        0.21157574],
       [0.10022164, 0.25203508, 0.56541056, ..., 0.32013739, 0.27361781,
        0.21676032]])

In [20]:
X_val

array([[0.09279806, 0.48290415, 0.40115905, ..., 0.25013934, 0.35515309,
        0.19119137],
       [0.22587376, 0.57586514, 0.5234926 , ..., 0.09718426, 0.26846398,
        0.20508329],
       [0.27062883, 0.50147351, 0.64199167, ..., 0.13470006, 0.27298138,
        0.20555256],
       ...,
       [0.17386564, 0.28748482, 0.58560038, ..., 0.32121816, 0.27770392,
        0.21623285],
       [0.2436465 , 0.21720261, 0.58267034, ..., 0.32175066, 0.27582463,
        0.21706811],
       [0.17195916, 0.50168494, 0.42404601, ..., 0.39851863, 0.13237835,
        0.20340643]])

**Machine learning models (SVM, Random Forest, Logistic Regression)**

In [21]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import GridSearchCV as Grid
from sklearn.metrics import accuracy_score as Score
from sklearn.metrics import classification_report as Report

SVM

In [22]:
grid_parameter= {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

svm_mod = SVC()

grid_var = Grid(svm_mod, grid_parameter, cv=2, verbose=1)
grid_var.fit(X_trn, y_trn)

print(f"Best Parameters: {grid_var.best_params_}")
svm_mod = grid_var.best_estimator_

Y_pred = svm_mod.predict(X_val)
accuracy = Score(y_val, Y_pred)
Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Validation Score: {accuracy:.4f}")
print(Report(y_val, Y_pred))
print("Macro_F1_score", Macro_F1_score)

Fitting 2 folds for each of 18 candidates, totalling 36 fits
Best Parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}


, Validation Score: 0.5910
              precision    recall  f1-score   support

           0       0.59      0.90      0.71      3015
           1       0.59      0.19      0.29      2305

    accuracy                           0.59      5320
   macro avg       0.59      0.54      0.50      5320
weighted avg       0.59      0.59      0.53      5320

Macro_F1_score 0.4999321799606903


In [23]:
Y_pred = svm_mod.predict(X_tst)
accuracy = Score(y_tst, Y_pred)
Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Testing Score: {accuracy:.4f}")
print(Report(y_tst, Y_pred))
print("Macro_F1_score", Macro_F1_score)



, Testing Score: 0.5857
              precision    recall  f1-score   support

           0       0.59      0.89      0.71      2986
           1       0.58      0.20      0.30      2334

    accuracy                           0.59      5320
   macro avg       0.58      0.54      0.50      5320
weighted avg       0.58      0.59      0.53      5320

Macro_F1_score 0.5017419281145293


Random Forest

In [24]:
grid_parameter= {
    'n_estimators': [50, 10, 20],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

RF_mod = RF()

grid_var = Grid(RF_mod, grid_parameter, cv=2, verbose=1)
grid_var.fit(X_trn, y_trn)

print(f"Best Parameters: {grid_var.best_params_}")
RF_mod = grid_var.best_estimator_

Y_pred = RF_mod.predict(X_val)
accuracy = Score(y_val, Y_pred)
Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Validation Score: {accuracy:.4f}")
print(Report(y_val, Y_pred))
print("Macro_F1_score", Macro_F1_score)


Fitting 2 folds for each of 36 candidates, totalling 72 fits
Best Parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 50}


, Validation Score: 0.5861
              precision    recall  f1-score   support

           0       0.60      0.79      0.68      3015
           1       0.54      0.32      0.40      2305

    accuracy                           0.59      5320
   macro avg       0.57      0.55      0.54      5320
weighted avg       0.57      0.59      0.56      5320

Macro_F1_score 0.5418005713998734


In [25]:
Y_pred = RF_mod.predict(X_tst)
accuracy = Score(y_tst, Y_pred)
Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Testing Score: {accuracy:.4f}")
print(Report(y_tst, Y_pred))
print("Macro_F1_score", Macro_F1_score)



, Testing Score: 0.5867
              precision    recall  f1-score   support

           0       0.60      0.79      0.68      2986
           1       0.55      0.33      0.41      2334

    accuracy                           0.59      5320
   macro avg       0.57      0.56      0.55      5320
weighted avg       0.58      0.59      0.56      5320

Macro_F1_score 0.5453785016945888


Logistic Regression

In [26]:
grid_parameter= {
    'C': [0.01, 0.1, 1],
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'max_iter': [10, 20, 30]
}

LR_mod = LR()

grid_var = Grid(LR_mod, grid_parameter, cv=2, verbose=1)
grid_var.fit(X_trn, y_trn)

print(f"Best Parameters: {grid_var.best_params_}")
LR_mod = grid_var.best_estimator_

Y_pred = LR_mod.predict(X_val)
accuracy = Score(y_val, Y_pred)
Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Validation Score: {accuracy:.4f}")
print(Report(y_val, Y_pred))
print("Macro_F1_score", Macro_F1_score)


Fitting 2 folds for each of 27 candidates, totalling 54 fits
Best Parameters: {'C': 1, 'max_iter': 10, 'solver': 'newton-cg'}


, Validation Score: 0.5930
              precision    recall  f1-score   support

           0       0.60      0.88      0.71      3015
           1       0.58      0.22      0.32      2305

    accuracy                           0.59      5320
   macro avg       0.59      0.55      0.51      5320
weighted avg       0.59      0.59      0.54      5320

Macro_F1_score 0.5136926522193129


In [27]:
Y_pred = LR_mod.predict(X_tst)
accuracy = Score(y_tst, Y_pred)
Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Testing Score: {accuracy:.4f}")
print(Report(y_tst, Y_pred))
print("Macro_F1_score", Macro_F1_score)



, Testing Score: 0.5850
              precision    recall  f1-score   support

           0       0.59      0.87      0.70      2986
           1       0.57      0.21      0.31      2334

    accuracy                           0.58      5320
   macro avg       0.58      0.54      0.51      5320
weighted avg       0.58      0.58      0.53      5320

Macro_F1_score 0.5073135245846632


**Deep learning models (LSTM, BiLSTM)**

In [28]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

LSTM

In [29]:
# hyper tuning the optimizer
for optimizer in ['adam', 'adamax', 'adagrad']:
  print("Result of hyper tuning the optimizer ", optimizer)

  LSTM_mod = Sequential()
  LSTM_mod.add(LSTM(200, input_shape=(X_trn.shape[1], 1)))
  LSTM_mod.add(Dense(1, activation='sigmoid'))
  LSTM_mod.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  LSTM_mod.fit(X_trn, y_trn, epochs=20, batch_size=128)

  Y_pred = LSTM_mod.predict(X_val)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_val, Y_pred)
  Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']
  print(f"\n\n Validation Score: {accuracy:.4f}")
  print(Report(y_val, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

  Y_pred = LSTM_mod.predict(X_tst)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_tst, Y_pred)
  Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

  print(f"\n\n Testing Score: {accuracy:.4f}")
  print(Report(y_tst, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

Result of hyper tuning the optimizer  adam
Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.5553 - loss: 0.6870
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 89ms/step - accuracy: 0.5528 - loss: 0.6872
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 17s 59ms/step - accuracy: 0.5658 - loss: 0.6791
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - accuracy: 0.5704 - loss: 0.6785
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.5704 - loss: 0.6772
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.5668 - loss: 0.6800
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.5711 - loss: 0.6755
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.5690 - loss: 0.6795
Epoch 9/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.5749 - loss: 0.6781
Epoch 10/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.5696 - loss: 0.6777
Epoch 11/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.5755 - loss: 0.6

In [30]:
# hyper tuning the count of epoches
for Epoches in [5, 10, 20]:
  print("Result of hyper tuning the count of epoches ", Epoches)

  LSTM_mod = Sequential()
  LSTM_mod.add(LSTM(200, input_shape=(X_trn.shape[1], 1)))
  LSTM_mod.add(Dense(1, activation='sigmoid'))
  LSTM_mod.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
  LSTM_mod.fit(X_trn, y_trn, epochs=Epoches, batch_size=128)

  Y_pred = LSTM_mod.predict(X_val)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_val, Y_pred)
  Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']
  print(f"\n\n Validation Score: {accuracy:.4f}")
  print(Report(y_val, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

  Y_pred = LSTM_mod.predict(X_tst)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_tst, Y_pred)
  Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

  print(f"\n\n Testing Score: {accuracy:.4f}")
  print(Report(y_tst, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

Result of hyper tuning the count of epoches  5
Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - accuracy: 0.5597 - loss: 0.6874
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 49ms/step - accuracy: 0.5602 - loss: 0.6863
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 62ms/step - accuracy: 0.5613 - loss: 0.6858
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 45ms/step - accuracy: 0.5630 - loss: 0.6856
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - accuracy: 0.5617 - loss: 0.6853
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step


 Validation Score: 0.5667
              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3015
           1       0.00      0.00      0.00      2305

    accuracy                           0.57      5320
   macro avg       0.28      0.50      0.36      5320
weighted avg       0.32      0.57      0.41      5320

Macro_F1_score 0.36172765446910615
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step


 Testing Score: 0.5613
              precisi

BiLSTM

In [31]:
X_trn = np.expand_dims(X_trn, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_tst = np.expand_dims(X_tst, axis=-1)

In [32]:
# hyper tuning the optimizer
for optimizer in ['adam', 'adamax', 'adagrad']:
  print("Result of hyper tuning the optimizer ", optimizer)

  bi_LSTM_mod = Sequential()
  bi_LSTM_mod.add(Bidirectional(LSTM(200, input_shape=(X_trn.shape[1], 1))))
  bi_LSTM_mod.add(Dense(1, activation='sigmoid'))
  bi_LSTM_mod.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  bi_LSTM_mod.fit(X_trn, y_trn, epochs=20, batch_size=128)

  Y_pred = bi_LSTM_mod.predict(X_val)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_val, Y_pred)
  Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']
  print(f"\n\n Validation Score: {accuracy:.4f}")
  print(Report(y_val, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

  Y_pred = bi_LSTM_mod.predict(X_tst)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_tst, Y_pred)
  Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

  print(f"\n\n Testing Score: {accuracy:.4f}")
  print(Report(y_tst, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

Result of hyper tuning the optimizer  adam
Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 94ms/step - accuracy: 0.5654 - loss: 0.6853
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 90ms/step - accuracy: 0.5539 - loss: 0.6857
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 94ms/step - accuracy: 0.5682 - loss: 0.6804
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 91ms/step - accuracy: 0.5758 - loss: 0.6792
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 97ms/step - accuracy: 0.5707 - loss: 0.6793
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 91ms/step - accuracy: 0.5777 - loss: 0.6777
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 95ms/step - accuracy: 0.5774 - loss: 0.6766
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 91ms/step - accuracy: 0.5753 - loss: 0.6768
Epoch 9/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 95ms/step - accuracy: 0.5735 - loss: 0.6790
Epoch 10/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 93ms/step - accuracy: 0.5781 - loss: 0.6755
Epoch 11/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 95ms/step - accuracy: 0.5698 - 

In [33]:
# hyper tuning the count of epoches
for Epoches in [5, 10, 20]:
  print("Result of hyper tuning the count of epoches ", Epoches)

  bi_LSTM_mod = Sequential()
  bi_LSTM_mod.add(Bidirectional(LSTM(200, input_shape=(X_trn.shape[1], 1))))
  bi_LSTM_mod.add(Dense(1, activation='sigmoid'))
  bi_LSTM_mod.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  bi_LSTM_mod.fit(X_trn, y_trn, epochs=Epoches, batch_size=128)

  Y_pred = bi_LSTM_mod.predict(X_val)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_val, Y_pred)
  Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']
  print(f"\n\n Validation Score: {accuracy:.4f}")
  print(Report(y_val, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

  Y_pred = bi_LSTM_mod.predict(X_tst)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_tst, Y_pred)
  Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

  print(f"\n\n Testing Score: {accuracy:.4f}")
  print(Report(y_tst, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

Result of hyper tuning the count of epoches  5
Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 85ms/step - accuracy: 0.5514 - loss: 0.6881
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 89ms/step - accuracy: 0.5623 - loss: 0.6851
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 95ms/step - accuracy: 0.5635 - loss: 0.6834
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 96ms/step - accuracy: 0.5695 - loss: 0.6797
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 85ms/step - accuracy: 0.5703 - loss: 0.6797
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step


 Validation Score: 0.5833
              precision    recall  f1-score   support

           0       0.60      0.81      0.69      3015
           1       0.54      0.28      0.37      2305

    accuracy                           0.58      5320
   macro avg       0.57      0.55      0.53      5320
weighted avg       0.57      0.58      0.55      5320

Macro_F1_score 0.5291939741471909
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step


 Testing Score: 0.5870
              precis